In [2]:
import pandas as pd

gpt_4_annotation = pd.read_csv("../data/lmsys_data/lmsys_user_prompts_jaccard0.9_w_factually_w_topics_w_model_v1.1_prompts_evaluated_gpt4.csv")
llama_3_annotations = pd.read_csv("../data/lmsys_data/lmsys_user_prompts_jaccard0.9_w_factually_w_topics_w_model_v1.1_prompts_evaluated_llama3_instruct.csv")
# df = pd.read_csv("final_dataset_w_avg_scors.csv")
gpt_4_annotation.columns, llama_3_annotations.columns


(Index(['conversation_hash', 'user_prompts', 'model_responses', 'embedding',
        'embedding_dim10_Nnbr15', 'Topics', 'model', 'factual_vs_faithful',
        'scores'],
       dtype='object'),
 Index(['conversation_hash', 'user_prompts', 'model_responses', 'embedding',
        'embedding_dim10_Nnbr15', 'Topics', 'model', 'factual_vs_faithful',
        'scores'],
       dtype='object'))

In [7]:
import re
import ast
pattern = r'(\{.*"Clarity".*"Generalizability".*"Relevance".*"Actionability".*"Feasibility".*\})'
def extract_valid_json(val):
    if isinstance(val, str):
        match = re.search(pattern, val, re.DOTALL)  # Extract the dictionary portion
        if match:
            try:
                # Clean up the extracted match by removing newlines and unnecessary characters
                json_str = match.group(1).replace("\n", "").replace("\r", "").strip()
                return ast.literal_eval(json_str)
            except (ValueError, SyntaxError):
                return None
    return None

# extract_valid_json(x)
# gpt_4_annotation['scores_gpt4'] = gpt_4_annotation['scores'].apply(extract_valid_json)
# gpt_4_annotation['scores_llama3'] = llama_3_annotations['scores'].apply(extract_valid_json)

# non_matching_rows_llama3 = gpt_4_annotation[gpt_4_annotation['scores_llama3'].isnull() & llama_3_annotations['scores'].notnull()]
# print(len(non_matching_rows_llama3))

In [18]:
df_scores = pd.DataFrame({'user_prompts': gpt_4_annotation['user_prompts'],
                        'scores_gpt4': gpt_4_annotation['scores_gpt4'],
                        'scores_llama3': gpt_4_annotation['scores_llama3']})

df_scores = df_scores.groupby('user_prompts').first().reset_index()
# df_scores_unique.head(5)

# df_scores['GPT-4 Scores'] = df_scores['GPT-4 Scores'].apply(extract_valid_json)
# df_scores['LLAMA-3 Scores'] = df_scores['LLAMA-3 Scores'].apply(extract_valid_json)

In [19]:
def combine_eval_scores(row):
    overall_score = 0
    for k in ['Clarity', 'Generalizability', 'Relevance', 'Feasibility']: #actionability has been removed from the evaluation
        if row['scores_gpt4'] is None and row['scores_llama3'] is None:
            overall_score += 0
        elif row['scores_llama3'] is None:
            overall_score += row['scores_gpt4'][k]
        elif row['scores_gpt4'] is None:
            overall_score += row['scores_llama3'][k]
        else:
            overall_score += (row['scores_gpt4'][k] + row['scores_llama3'][k])/2
    return overall_score/4
df_scores['prompt_score'] = df_scores.apply(combine_eval_scores, axis=1)

In [21]:
df_scores.columns
# df_scores.to_csv("../data/lmsys_data/lmsys_user_prompts_evaluated_avg_scores_comb.csv", index=False)
# df_unique_user_prompts.head(50)

Index(['user_prompts', 'scores_gpt4', 'scores_llama3', 'prompt_score'], dtype='object')

In [8]:
sorted_df = df.groupby('Topics')['comb_avg_score'].mean().sort_values(ascending=False)
print(sorted_df)
print(len(df), len(df[(df['comb_avg_score'] >= 4) & (df['Topics'] != -1)]))

# index_of_topic_minus_1 = sorted_df.index.tolist().index(-1)
# print(index_of_topic_minus_1)


Topics
168    4.889474
365    4.800000
285    4.700000
91     4.700000
266    4.700000
         ...   
186    2.786667
377    2.750000
310    2.700000
167    2.000000
324    1.875000
Name: comb_avg_score, Length: 298, dtype: float64
20824 5610


In [13]:
len(df[(df['comb_avg_score'] >= 4.0) & (df['Topics'] != -1) & (df['model'].isin(['gpt-4', 'gpt-3.5-turbo', 'claude-2', 'palm-2', 'claude-1', 'claude-instant-1']))])

2688

In [29]:
input_file_path = "../data/lmsys_data/top_4000_data_instances.csv"
# input_file_path = "../data/lmsys_data/top_3000_data_noise_excluded.csv"
df_final = pd.read_csv(input_file_path)

# df_complete = pd.read_parquet(input_file_path)

# df_s = df_complete[df_complete['user_prompts'] == "what kind of poem rhymes on every even numbered line?"]

In [30]:
len(df_final[df_final["Topics"] != -1])
len(df_final.groupby('Topics').first())

242